In [2]:
import ROOT

# Open the ROOT file
file = ROOT.TFile("/scratch/cms/store/mc/RunIISummer20UL17NanoAODv9/ZGToLLG_01J_5f_TuneCP5_13TeV-amcatnloFXFX-pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/70000/AF4D5462-30E8-EA43-AA8A-A1EADA9796BC.root")

# Access the TTree in the ROOT file (replace 'tree_name' with the actual tree name)
tree = file.Get("Events")

# Browse the branches
for branch in tree.GetListOfBranches():
    print(branch.GetName())  # Print the name of each branch

run
luminosityBlock
event
HTXS_Higgs_pt
HTXS_Higgs_y
HTXS_stage1_1_cat_pTjet25GeV
HTXS_stage1_1_cat_pTjet30GeV
HTXS_stage1_1_fine_cat_pTjet25GeV
HTXS_stage1_1_fine_cat_pTjet30GeV
HTXS_stage1_2_cat_pTjet25GeV
HTXS_stage1_2_cat_pTjet30GeV
HTXS_stage1_2_fine_cat_pTjet25GeV
HTXS_stage1_2_fine_cat_pTjet30GeV
HTXS_stage_0
HTXS_stage_1_pTjet25
HTXS_stage_1_pTjet30
HTXS_njets25
HTXS_njets30
nboostedTau
boostedTau_chargedIso
boostedTau_eta
boostedTau_leadTkDeltaEta
boostedTau_leadTkDeltaPhi
boostedTau_leadTkPtOverTauPt
boostedTau_mass
boostedTau_neutralIso
boostedTau_phi
boostedTau_photonsOutsideSignalCone
boostedTau_pt
boostedTau_puCorr
boostedTau_rawAntiEle2018
boostedTau_rawIso
boostedTau_rawIsodR03
boostedTau_rawMVAnewDM2017v2
boostedTau_rawMVAoldDM2017v2
boostedTau_rawMVAoldDMdR032017v2
boostedTau_charge
boostedTau_decayMode
boostedTau_jetIdx
boostedTau_rawAntiEleCat2018
boostedTau_idAntiEle2018
boostedTau_idAntiMu
boostedTau_idMVAnewDM2017v2
boostedTau_idMVAoldDM2017v2
boostedTau_idMVAold

Warning in <TClass::Init>: no dictionary for class edm::Hash<1> is available
Warning in <TClass::Init>: no dictionary for class edm::ProcessHistory is available
Warning in <TClass::Init>: no dictionary for class edm::ProcessConfiguration is available
Warning in <TClass::Init>: no dictionary for class edm::ParameterSetBlob is available
Warning in <TClass::Init>: no dictionary for class pair<edm::Hash<1>,edm::ParameterSetBlob> is available
